# Psoriasis - Infertility

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import xgboost as xgb

from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score

#import prince 
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report
from scipy.stats import chi2_contingency

In [2]:
def days_between(d1, d2):
    
    if pd.isna(d1) or pd.isna(d2):
        return np.nan
    else:
        d1_ = d1.date()
        d2_ = d2.date()
        
        return (d2_ - d1_).days

In [3]:
def months_between(d1, d2):
    if pd.isnull(d1) or pd.isnull(d2):
        return np.nan
    else:
        return (d2 - d1).days / 30.44  # Average days in a month

In [4]:
def create_table(df1, df2, col, n_cols):
    
    y1 = df1[col].value_counts(sort=False).sort_index()
    y1_indeces = y1.index.to_numpy()
    y1_values  = y1.values
    
    y2 = df2[col].value_counts(sort=False).sort_index()
    y2_indeces = y2.index.to_numpy()
    y2_values  = y2.values
    
    table = np.zeros((2, n_cols), dtype=int)
    
    for idx, val in enumerate(y1_indeces):
        table[0,val+1] = y1_values[idx]   
    
    for idx, val in enumerate(y2_indeces):
        table[1,val+1] = y2_values[idx]
    
    return table


def run_chi2(dataframe1, dataframe2, feature_list, label1="INF", label2="INF+PSO", columns=[-1,0,1], verbose=True):

    important_features = {}

    for feature in feature_list:

        if verbose:
            print("* Testing feature %s"%feature)
    
        table = create_table(dataframe1, dataframe2, feature, len(columns))
    
        df = pd.DataFrame(data=table, index=[label1, label2], columns=columns)
        
        if verbose:
            display(df)
    
        try:
            res = chi2_contingency(table, correction=True) 
        
            if res.pvalue <= 0.05:
                if verbose:
                    print(res.pvalue, res.pvalue <= 0.05)
                important_features[feature] = {"p-val": res.pvalue, "table": df}
        
        except:
            pass
    
        if verbose:
            print()
            print()

    return important_features

In [5]:
# Define the adjusted create_table function
def create_table_month(df1, df2, col, categories):
    y1 = df1[col].value_counts(sort=False)
    y1 = y1.reindex(categories, fill_value=0)
    y1_values  = y1.values

    y2 = df2[col].value_counts(sort=False)
    y2 = y2.reindex(categories, fill_value=0)
    y2_values  = y2.values

    table = np.array([y1_values, y2_values])

    return table, categories

# Define the adjusted run_chi2 function
def run_chi2_month(dataframe1, dataframe2, feature_list, label1="INF", label2="INF+PSO", categories=None, verbose=False):
    important_features = {}

    for feature in feature_list:
        table, columns = create_table_month(dataframe1, dataframe2, feature, categories)
        df = pd.DataFrame(data=table, index=[label1, label2], columns=columns)

        # Perform the chi-squared test
        try:
            chi2, p_value, dof, expected = chi2_contingency(table)
            # Store only if p-value <= 0.05
            if p_value <= 0.05:
                if verbose:
                    print(f"* Significant feature: {feature}")
                    print(f"P-value: {p_value}")
                    display(df)
                important_features[feature] = {"p-val": p_value, "table": df}
        except Exception as e:
            if verbose:
                print(f"An error occurred with feature {feature}: {e}")

    return important_features

In [6]:
df = pd.read_excel("/Users/kryptonempyrean/Desktop/Tesi Material/OneDrive_1_08-09-2024/Psoriasis_2017_Erez_Data2_Coded codifica in corso.xlsx", sheet_name="PsoriasisPanel")

In [7]:
df["sex"].value_counts()

sex
F    141769
M    138684
Name: count, dtype: int64

In [8]:
df = df[df["sex"] == "M"]

In [9]:
columns = pd.read_excel("/Users/kryptonempyrean/Desktop/Tesi Material/OneDrive_1_08-09-2024/Psoriasis_2017_Erez_Data2_Coded codifica in corso.xlsx", sheet_name="Foglio1", header=None)
columns = columns[1].tolist()

comorbidities = columns[:142]
selected_comorbidities = sorted(set(df.columns.tolist()) & set(comorbidities),  key = df.columns.tolist().index)
selected_comorbidities.remove("Psoriasis")

selected_columns = ['date_of_birth', "Infertility ", "Psoriasis"] + selected_comorbidities

In [10]:
data = df[selected_columns]
data

,date_of_birth,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,Syphilis / Gonorrhea,Hepatitis B Carrier,Hepatitis C Carrier,Familial Mediteranean Fever,Amyloidosis,...,OncBone,OncSaracoma,OncGenitalia,Oncmyeloma,OncPolycythemiaVera,OncMyelodysplastic,OncLympholiferative,OncNeurofibromatosis,OncOther,OncUnKnow
0,1906-05-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1906-06-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1907-06-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1907-05-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138679,2016-01-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138680,2016-05-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138681,2016-06-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138682,2016-10-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_subset = data.dropna(axis='columns', how='all').copy()

In [12]:
for var in data_subset.columns:
    if var not in ['GroupName', 'date_of_birth']:
        data_subset["%s_binary"%var] = data_subset["%s"%var].fillna(False)
        data_subset["%s_binary"%var] = data_subset["%s_binary"%var] != False

In [13]:
patients_pso = data_subset[data_subset["Psoriasis_binary"]]

In [14]:
patients_pso[["Infertility ", "Psoriasis"]].head(10)

,Infertility,Psoriasis
7,NaN,1999-11-16 00:00:00
8,NaN,2004-02-23 00:00:00
16,NaN,2004-02-06 00:00:00
25,NaN,2007-04-08 00:00:00
31,NaN,2000-11-17 00:00:00
36,NaN,2006-12-06 00:00:00
41,NaN,2005-09-03 00:00:00
54,NaN,2004-05-09 00:00:00
61,NaN,2009-10-25 00:00:00
63,NaN,2002-02-01 00:00:00


In [15]:
values = patients_pso.apply(lambda x: days_between(x["Psoriasis"], x['Infertility ']), axis=1)
indeces = patients_pso[pd.isna(values)].index.tolist() + patients_pso[values > 0].index.tolist()
filtered_pso = patients_pso[patients_pso.index.isin(indeces)]
filtered_pso.head(10)

,date_of_birth,Infertility,Psoriasis,Tuberculosis,Tuberculosis s/p,Syphilis / Gonorrhea,Hepatitis B Carrier,Hepatitis C Carrier,Familial Mediteranean Fever,Amyloidosis,...,OncBone_binary,OncSaracoma_binary,OncGenitalia_binary,Oncmyeloma_binary,OncPolycythemiaVera_binary,OncMyelodysplastic_binary,OncLympholiferative_binary,OncNeurofibromatosis_binary,OncOther_binary,OncUnKnow_binary
7,1909-05-01 00:00:00,NaN,1999-11-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
8,1909-06-09 00:00:00,NaN,2004-02-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
16,1910-06-21 00:00:00,NaN,2004-02-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
25,1911-07-27 00:00:00,NaN,2007-04-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
31,1911-05-02 00:00:00,NaN,2000-11-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
36,1911-01-01 00:00:00,NaN,2006-12-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
41,1912-12-26 00:00:00,NaN,2005-09-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
54,1912-01-01 00:00:00,NaN,2004-05-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
61,1913-02-08 00:00:00,NaN,2009-10-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
63,1913-09-05 00:00:00,NaN,2002-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [16]:
for var in filtered_pso.columns[3:123]:
    values_pso = filtered_pso.apply(lambda x: days_between(x["Psoriasis"], x[var]), axis=1)
    values_inf = filtered_pso.apply(lambda x: days_between(x["Infertility "], x[var]), axis=1)
    df = pd.DataFrame([values_pso.fillna(0), values_inf.fillna(0)], index=["pso", "inf"]).T
    df["group"] = 0
    df["group"][df["pso"] < 0] = -1
    df["group"][(df["pso"] > 0) & (df["inf"] <= 0)] = 1
    
    filtered_pso[var+"_feature"] = df["group"].astype("category")

In [17]:
feature_list = filtered_pso.columns[filtered_pso.columns.str.contains("_feature")]

In [18]:
df = filtered_pso[feature_list]
df.head(10)

,Tuberculosis_feature,Tuberculosis s/p_feature,Syphilis / Gonorrhea_feature,Hepatitis B Carrier_feature,Hepatitis C Carrier_feature,Familial Mediteranean Fever_feature,Amyloidosis_feature,Malignancy_feature,Benign Brain Tumor_feature,Hyperthyroidism_feature,...,OncBone_feature,OncSaracoma_feature,OncGenitalia_feature,Oncmyeloma_feature,OncPolycythemiaVera_feature,OncMyelodysplastic_feature,OncLympholiferative_feature,OncNeurofibromatosis_feature,OncOther_feature,OncUnKnow_feature
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
patients_only_pso = filtered_pso[filtered_pso["Infertility _binary"]==0]
patients_pso_inf  = filtered_pso[filtered_pso["Infertility _binary"]==1]

In [20]:
dictionary_test1 = run_chi2(patients_only_pso,
                            patients_pso_inf,
                            feature_list,
                            label1="PSO",
                            label2="PSO+INF",
                            verbose=False)

for feature in dictionary_test1:
    print(feature)
    display(dictionary_test1[feature]["table"])

Hepatitis C Carrier_feature


,-1,0,1
PSO,123,14483,86
PSO+INF,1,39,2


Hyperthyroidism_feature


,-1,0,1
PSO,72,14539,81
PSO+INF,0,40,2


Obesity_feature


...

,-1,0,1
PSO,9,14656,27
PSO+INF,0,41,1


OnclymphomaNonHodgkin_feature


,-1,0,1
PSO,67,14518,107
PSO+INF,0,40,2


OncSaracoma_feature


,-1,0,1
PSO,14,14646,32
PSO+INF,0,41,1


In [21]:
#One-Hot Encoding
comorbidity_cols = df.columns
data_encoded = df[comorbidity_cols].astype(str)
data_ohe = pd.get_dummies(data_encoded, columns=comorbidity_cols, prefix_sep='_')

In [22]:
data_ohe.head(10)

,Tuberculosis_feature_-1,Tuberculosis_feature_0,Tuberculosis_feature_1,Tuberculosis s/p_feature_-1,Tuberculosis s/p_feature_0,Tuberculosis s/p_feature_1,Syphilis / Gonorrhea_feature_-1,Syphilis / Gonorrhea_feature_0,Syphilis / Gonorrhea_feature_1,Hepatitis B Carrier_feature_-1,...,OncLympholiferative_feature_1,OncNeurofibromatosis_feature_-1,OncNeurofibromatosis_feature_0,OncNeurofibromatosis_feature_1,OncOther_feature_-1,OncOther_feature_0,OncOther_feature_1,OncUnKnow_feature_-1,OncUnKnow_feature_0,OncUnKnow_feature_1
7,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
8,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
16,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
25,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
31,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
36,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
41,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
54,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
61,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False
63,False,True,False,False,True,False,False,True,False,False,...,False,False,True,False,False,True,False,False,True,False


In [23]:
import prince

for n in range(10,80,10):
    mca = prince.MCA(n_components=n, random_state=42)
    data_mca = mca.fit_transform(data_ohe)
    
    silhouette = {}
    print(f"Silhouette Scores with MCA number of components of {n}")
    for n_clusters in range(2, 10):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        silhouette[n_clusters] = silhouette_score(data_mca, kmeans.fit_predict(data_mca))
        
        
    print(silhouette)
    print("\n\n")

Silhouette Scores with MCA number of components of 10
{2: 0.4678841052732825, 3: 0.4420466167420275, 4: 0.31135378365314637, 5: 0.31460959859703513, 6: 0.3102281726915142, 7: 0.3137654438048272, 8: 0.3174270953303729, 9: 0.3196920402533872}



Silhouette Scores with MCA number of components of 20
{2: 0.4416435200450966, 3: 0.40382688723134713, 4: 0.26071777569275845, 5: 0.2669034391482084, 6: 0.266097119314136, 7: 0.24498763684906683, 8: 0.25199520829162264, 9: 0.25391300355164537}



Silhouette Scores with MCA number of components of 30
{2: 0.42894125577756903, 3: 0.2642451072498205, 4: 0.24041959470408672, 5: 0.2196357987716409, 6: 0.26020598825275737, 7: 0.24139434451995315, 8: 0.2260605353595839, 9: 0.22027892115028302}



Silhouette Scores with MCA number of components of 40
{2: 0.4203785358282746, 3: 0.3731657177954837, 4: 0.36406354404901003, 5: 0.20433506758307338, 6: 0.21891465175652441, 7: 0.24434934051634716, 8: 0.2334020066595503, 9: 0.20582801857939578}



Silhouette Score

In [24]:
for n in range(10,80,10):
    mca = prince.MCA(n_components=n, random_state=42)
    data_mca = mca.fit_transform(data_ohe)
    
    silhouette = {}
    print(f"Silhouette Scores with MCA number of components of {n}")
    for n_clusters in range(2, 10):
        agglo = AgglomerativeClustering(n_clusters=n_clusters)
        silhouette[n_clusters] = silhouette_score(data_mca, agglo.fit_predict(data_mca))
        
        
    print(silhouette)
    print("\n\n")

Silhouette Scores with MCA number of components of 10
{2: 0.4507288567695536, 3: 0.2126356994206525, 4: 0.22490416012407116, 5: 0.23241880957009922, 6: 0.23976138338667455, 7: 0.2443544342247394, 8: 0.24956482511664752, 9: 0.25431612968863176}



Silhouette Scores with MCA number of components of 20
{2: 0.24787855191377745, 3: 0.24765571626830532, 4: 0.249277397482181, 5: 0.25110089123751766, 6: 0.24838041207599607, 7: 0.24995329370266892, 8: 0.2544109239948988, 9: 0.2532549179648685}



Silhouette Scores with MCA number of components of 30
{2: 0.25888219363169407, 3: 0.23456078907808914, 4: 0.23720363942685851, 5: 0.237292670349379, 6: 0.24167421191458893, 7: 0.24488878845144788, 8: 0.2424809943124632, 9: 0.24444267942500653}



Silhouette Scores with MCA number of components of 40
{2: 0.2358438613656777, 3: 0.23664255109130247, 4: 0.22544265155621176, 5: 0.22629776140991656, 6: 0.2258692645952561, 7: 0.2300983497642592, 8: 0.23247412358231095, 9: 0.2317386270198962}



Silhouette Sco

In [25]:
for n in range(10,80,10):
    mca = prince.MCA(n_components=n, random_state=42)
    data_mca = mca.fit_transform(data_ohe)
    
    silhouette = {}
    print(f"Silhouette Scores with MCA number of components of {n}")
    for n_clusters in range(2, 10):
        spectral = SpectralClustering(n_clusters=n_clusters, random_state=42)
        silhouette[n_clusters] = silhouette_score(data_mca, spectral.fit_predict(data_mca))
        
        
    print(silhouette)
    print("\n\n")

Silhouette Scores with MCA number of components of 10
{2: 0.49078929489599993, 3: 0.8271535412354806, 4: 0.799248622965602, 5: 0.8022406595947651, 6: 0.7957814239623768, 7: 0.7670260798437367, 8: 0.7587204331752633, 9: 0.7588318869461983}



Silhouette Scores with MCA number of components of 20
{2: 0.8338579667343253, 3: 0.8330964824398552, 4: 0.8329167874729128, 5: 0.8319147367157722, 6: 0.8197769162266577, 7: 0.8312229854478398, 8: 0.8193706703226098, 9: 0.817352209045706}



Silhouette Scores with MCA number of components of 30
{2: 0.8489169019338123, 3: 0.8132195786863659, 4: 0.8127615381700896, 5: 0.8343490181894904, 6: 0.8247629099255269, 7: 0.8243745461999116, 8: 0.824218770790478, 9: 0.8165295059414102}



Silhouette Scores with MCA number of components of 40
{2: 0.8430243959870206, 3: 0.8213766189087908, 4: 0.7954405431627444, 5: 0.7951370461156669, 6: 0.7950975170580631, 7: 0.820025159507279, 8: 0.8045979004669329, 9: 0.8041774988342378}



Silhouette Scores with MCA number o

In [26]:
for n in range(10,80,10):
    mca = prince.MCA(n_components=n, random_state=42)
    data_mca = mca.fit_transform(data_ohe)
    
    silhouette = {}
    print(f"Silhouette Scores with MCA number of components of {n}")
    for n_clusters in range(2, 10):
        gaussian = GaussianMixture(n_components=n_clusters, random_state=42)
        silhouette[n_clusters] = silhouette_score(data_mca, gaussian.fit_predict(data_mca))
        
        
    print(silhouette)
    print("\n\n")

Silhouette Scores with MCA number of components of 10
{2: 0.24723741968414747, 3: 0.1287085057332991, 4: 0.10833374850424049, 5: 0.11179555384810454, 6: 0.1207024736057225, 7: 0.12023222123878766, 8: 0.11955573629620687, 9: 0.10675197960640892}



Silhouette Scores with MCA number of components of 20
{2: 0.2512064482801552, 3: 0.17855376860916905, 4: 0.15218566547781498, 5: 0.14208944083643454, 6: 0.1391657778224057, 7: 0.14340451231975967, 8: 0.14225224058991984, 9: 0.14526253491382932}



Silhouette Scores with MCA number of components of 30
{2: 0.2776934600195571, 3: 0.21703330055082368, 4: 0.2177245313152547, 5: 0.18472308290789632, 6: 0.18368697502949002, 7: 0.18434808177226286, 8: 0.18638664233488597, 9: 0.18596466247033136}



Silhouette Scores with MCA number of components of 40
{2: 0.3213754008997037, 3: 0.3215215369133472, 4: 0.25271230284627555, 5: 0.2339325820119318, 6: 0.22233023373179991, 7: 0.22704381214025024, 8: 0.22180022996930038, 9: 0.22625073770755427}



Silhouett

In [27]:
mca = prince.MCA(n_components=30, random_state=42)
data_mca = mca.fit_transform(data_ohe)

In [28]:
data_mca

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
7,0.022661,-0.097291,-0.099517,-0.003423,0.082591,-0.001132,0.017638,0.013149,-0.062172,0.012817,...,-0.056378,-0.003824,-0.030377,0.071608,-0.008933,-0.015557,0.000693,0.001847,-0.032534,-0.016602
8,0.381437,0.141352,-0.155429,-0.043362,-0.080446,0.155814,-0.100857,-0.123383,-0.026325,-0.073753,...,-0.041488,0.012060,-0.044611,0.038659,-0.005366,-0.069772,-0.065515,-0.025422,0.049039,-0.028255
16,0.096011,-0.047081,0.040570,-0.112698,0.054469,-0.082967,0.072922,-0.167434,0.017892,-0.024795,...,0.011760,0.073487,0.035522,0.029357,0.082171,0.021921,0.055208,0.040845,-0.057754,-0.110739
25,0.195999,0.187829,-0.077066,-0.079158,-0.132518,0.226881,-0.038159,-0.018133,-0.052543,-0.016250,...,-0.075796,-0.052400,-0.266376,0.066768,-0.250546,0.010380,-0.061967,-0.024386,0.021323,-0.041365
31,-0.052855,0.011866,-0.006774,-0.050077,0.016186,0.020701,0.045968,0.005139,0.046767,-0.034554,...,-0.057317,0.048338,0.057482,0.104852,-0.093864,-0.086108,-0.048023,0.071090,0.012036,-0.018242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138362,-0.133184,0.024568,0.016730,-0.001924,0.005369,0.024241,0.023925,0.006729,-0.010798,0.000834,...,-0.010312,0.007077,-0.032809,0.011254,-0.028846,-0.030012,0.010405,-0.021943,-0.037459,0.005744
138417,-0.156755,0.000098,0.001314,-0.008722,0.028350,0.045518,0.005285,-0.030814,-0.025837,-0.000494,...,-0.007941,0.009996,-0.004953,0.007869,0.002266,-0.012075,-0.003896,0.003013,-0.001680,0.001090
138437,-0.156755,0.000098,0.001314,-0.008722,0.028350,0.045518,0.005285,-0.030814,-0.025837,-0.000494,...,-0.007941,0.009996,-0.004953,0.007869,0.002266,-0.012075,-0.003896,0.003013,-0.001680,0.001090
138562,-0.156755,0.000098,0.001314,-0.008722,0.028350,0.045518,0.005285,-0.030814,-0.025837,-0.000494,...,-0.007941,0.009996,-0.004953,0.007869,0.002266,-0.012075,-0.003896,0.003013,-0.001680,0.001090


In [30]:
spectral = SpectralClustering(n_clusters=2, random_state=42)
filtered_pso["Cluster"] = spectral.fit_predict(data_mca)
filtered_pso["Cluster"] = filtered_pso["Cluster"].astype("category")
filtered_pso["Cluster"].value_counts()

Cluster
0    14730
1        4
Name: count, dtype: int64

In [31]:
patients_group0 = filtered_pso[filtered_pso["Cluster"]==0]
patients_group1 = filtered_pso[filtered_pso["Cluster"]==1]

patients_group0_only_pso = patients_group0[patients_group0["Infertility _binary"] == 0]
patients_group1_only_pso = patients_group1[patients_group1["Infertility _binary"] == 0]

patients_group0_pso_inf = patients_group0[patients_group0["Infertility _binary"] == 1]
patients_group1_pso_inf = patients_group1[patients_group1["Infertility _binary"] == 1]

In [32]:
dictionary_test2 = run_chi2(patients_group0,
                            patients_group1,
                            feature_list,
                            label1="GROUP1",
                            label2="GROUP2",
                            verbose=False)

for feature in dictionary_test2:
    print(feature, "p-val: %2.3e"%dictionary_test2[feature]["p-val"])
    display(dictionary_test2[feature]["table"])

Hyperthyroidism_feature p-val: 1.831e-11


,-1,0,1
GROUP1,71,14576,83
GROUP2,1,3,0


Diabetes_feature p-val: 4.067e-02


,-1,0,1
GROUP1,1620,11171,1939
GROUP2,2,2,0


Other Endocrine and Metabolic Disease_feature p-val: 1.369e-40


...

,-1,0,1
GROUP1,193,14231,306
GROUP2,1,3,0


OncProatate_feature p-val: 6.448e-07


,-1,0,1
GROUP1,121,14358,251
GROUP2,1,3,0


OncLung_feature p-val: 6.559e-07


,-1,0,1
GROUP1,34,14575,121
GROUP2,0,3,1


In [33]:
dictionary_test3 = run_chi2(patients_group0_only_pso,
                            patients_group1_only_pso,
                            feature_list,
                            label1="GROUP1_PSO",
                            label2="GROUP2_PSO",
                            verbose=False)

for feature in dictionary_test3:
    print(feature, "p-val: %2.3e"%dictionary_test3[feature]["p-val"])
    display(dictionary_test3[feature]["table"])

Hyperthyroidism_feature p-val: 1.970e-11


,-1,0,1
GROUP1_PSO,71,14536,81
GROUP2_PSO,1,3,0


Diabetes_feature p-val: 4.096e-02


,-1,0,1
GROUP1_PSO,1618,11133,1937
GROUP2_PSO,2,2,0


Other Endocrine and Metabolic Disease_feature p-val: 1.786e-40


...

,-1,0,1
GROUP1_PSO,193,14191,304
GROUP2_PSO,1,3,0


OncProatate_feature p-val: 6.730e-07


,-1,0,1
GROUP1_PSO,121,14316,251
GROUP2_PSO,1,3,0


OncLung_feature p-val: 6.847e-07


,-1,0,1
GROUP1_PSO,34,14533,121
GROUP2_PSO,0,3,1


In [34]:
dictionary_test4 = run_chi2(patients_group0_only_pso,
                            patients_group0_pso_inf,
                            feature_list,
                            label1="GROUP1_PSO",
                            label2="GROUP1_PSO+INF",
                            verbose=False)

for feature in dictionary_test4:
    print(feature, "p-val: %2.3e"%dictionary_test4[feature]["p-val"])
    display(dictionary_test4[feature]["table"])

Hepatitis C Carrier_feature p-val: 1.141e-03


,-1,0,1
GROUP1_PSO,123,14479,86
GROUP1_PSO+INF,1,39,2


Hyperthyroidism_feature p-val: 1.208e-03


,-1,0,1
GROUP1_PSO,71,14536,81
GROUP1_PSO+INF,0,40,2


Obesity_feature p-val: 2.121e-02


...

,-1,0,1
GROUP1_PSO,9,14652,27
GROUP1_PSO+INF,0,41,1


OnclymphomaNonHodgkin_feature p-val: 8.857e-03


,-1,0,1
GROUP1_PSO,67,14514,107
GROUP1_PSO+INF,0,40,2


OncSaracoma_feature p-val: 1.225e-02


,-1,0,1
GROUP1_PSO,14,14642,32
GROUP1_PSO+INF,0,41,1


In [35]:
dictionary_test5 = run_chi2(patients_group1_only_pso,
                            patients_group1_pso_inf,
                            feature_list,
                            label1="GROUP2_PSO",
                            label2="GROUP2_PSO+INF",
                            verbose=False)

for feature in dictionary_test5:
    print(feature, "p-val: %2.3e"%dictionary_test5[feature]["p-val"])
    display(dictionary_test5[feature]["table"])

In [36]:
dictionary_test6 = run_chi2(patients_group0_pso_inf,
                            patients_group1_pso_inf,
                            feature_list,
                            label1="GROUP1_PSO+INF",
                            label2="GROUP2_PSO+INF",
                            verbose=False)

for feature in dictionary_test6:
    print(feature, "p-val: %2.3e"%dictionary_test6[feature]["p-val"])
    display(dictionary_test6[feature]["table"])

In [37]:
for var in patients_group0_pso_inf.columns[3:123]:
    values_pso = patients_group0_pso_inf.apply(lambda x: days_between(x["Psoriasis"], x[var]), axis=1)
    values_inf = patients_group0_pso_inf.apply(lambda x: days_between(x["Infertility "], x[var]), axis=1)
    df = pd.DataFrame([values_pso.fillna(0), values_inf.fillna(0)], index=["pso", "inf"]).T
    df["group"] = 0
    df["group"][df["pso"] < 0] = -1
    df["group"][(df["pso"] > 0) & (df["inf"] <= 0)] = 1
    df["group"][df["inf"] > 0] = 2
    
    patients_group0_pso_inf[var+"_feature"] = df["group"].astype("category")

In [38]:
for var in patients_group1_pso_inf.columns[3:123]:
    values_pso = patients_group1_pso_inf.apply(lambda x: days_between(x["Psoriasis"], x[var]), axis=1)
    values_inf = patients_group1_pso_inf.apply(lambda x: days_between(x["Infertility "], x[var]), axis=1)
    df = pd.DataFrame([values_pso.fillna(0), values_inf.fillna(0)], index=["pso", "inf"]).T
    df["group"] = 0
    df["group"][df["pso"] < 0] = -1
    df["group"][(df["pso"] > 0) & (df["inf"] <= 0)] = 1
    df["group"][df["inf"] > 0] = 2
    
    patients_group1_pso_inf[var+"_feature"] = df["group"].astype("category")

In [39]:
dictionary_test7 = run_chi2(patients_group0_pso_inf,
                            patients_group1_pso_inf,
                            feature_list,
                            label1="GROUP1_PSO+INF",
                            label2="GROUP2_PSO+INF",
                            columns=[-1,0,1,2],
                            verbose=False)

for feature in dictionary_test7:
    print(feature, "p-val: %2.3e"%dictionary_test7[feature]["p-val"])
    display(dictionary_test7[feature]["table"])

In [40]:
from stepmix.stepmix import StepMix

In [41]:
df = data_mca

silhouette = {}
for n_clusters in range(2, 6):
    model = StepMix(n_components=n_clusters, measurement="categorical", verbose=0, random_state=42)
    model.fit(df)
    silhouette[n_clusters] = silhouette_score(df, model.predict(df))

silhouette

Initializations (n_init) :   0%|          | 0/1 [00:00<?, ?it/s]

Fitting StepMix...


Initializations (n_init) :   0%|          | 0/1 [00:00<?, ?it/s]

Fitting StepMix...


Initializations (n_init) :   0%|          | 0/1 [00:00<?, ?it/s]

Fitting StepMix...


Initializations (n_init) :   0%|          | 0/1 [00:00<?, ?it/s]

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:21<00:00, 21.07s/it, max_LL=-962, max_avg_LL=-.0653]


{2: 0.827330495622287,
 3: 0.7493705327841992,
 4: 0.7018602903816366,
 5: 0.703640112520942}

In [42]:
from stepmix.stepmix import StepMixClassifier

In [48]:
df = data_mca
Y = 1*filtered_pso["Infertility _binary"].values

In [49]:
# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(df, Y, random_state=42)

# Fit StepMix
clf = StepMixClassifier(n_components=2, measurement='categorical', structural='binary', random_state=42, verbose=0, progress_bar=0)
clf.fit(X_train, Y_train)

StepMixClassifier(measurement='categorical', progress_bar=0, random_state=42,
                  structural='binary')

In [50]:
print(f'\nTest Accuracy: {balanced_accuracy_score(Y_test, clf.predict_class(X_test)):.4f}')


Test Accuracy: 0.4995


In [51]:
clf = SpectralClustering(n_components=2, random_state=42)
clf.fit(X_train, Y_train)

SpectralClustering(n_components=2, random_state=42)

In [53]:
print(f'\nTest Accuracy: {balanced_accuracy_score(Y_test, clf.fit_predict(X_test)):.4f}')


Test Accuracy: 0.4817


In [54]:
df = filtered_pso[feature_list]
Y = 1*filtered_pso["Infertility _binary"].values

In [55]:
# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(df, Y, random_state=42)

# Fit StepMix
clf = StepMixClassifier(n_components=2, measurement='categorical', structural='binary', random_state=42, verbose=0, progress_bar=0)
clf.fit(X_train, Y_train)

StepMixClassifier(measurement='categorical', progress_bar=0, random_state=42,
                  structural='binary')

In [56]:
print(f'\nTest Accuracy: {balanced_accuracy_score(Y_test, clf.predict_class(X_test)):.4f}')


Test Accuracy: 0.3843


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Assuming 'data_mca' is your feature matrix after MCA
# and 'psoriasis_status' is your target variable (0 or 1)

# Split data into features and target
X = data_mca
Y = 1*filtered_pso["Infertility _binary"].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba)}")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2934
           1       0.00      0.00      0.00        13

    accuracy                           1.00      2947
   macro avg       0.50      0.50      0.50      2947
weighted avg       0.99      1.00      0.99      2947

AUC-ROC: 0.6088039431597714


In [59]:
# Split data into features and target
X = filtered_pso[feature_list]
Y = 1*filtered_pso["Infertility _binary"].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:, 1]

# Evaluate the model
print(classification_report(y_test, y_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, y_proba)}")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2934
           1       0.00      0.00      0.00        13

    accuracy                           1.00      2947
   macro avg       0.50      0.50      0.50      2947
weighted avg       0.99      1.00      0.99      2947

AUC-ROC: 0.6775732788002727


In [61]:
from sklearn.model_selection import GridSearchCV

# Split data into features and target
X = data_mca
Y = 1*filtered_pso["Infertility _binary"].values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


In [62]:
best_model.score(X_test, y_test)

0.995588734306074

In [63]:
from sklearn.metrics import roc_auc_score

# Get predicted probabilities for the positive class
y_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
auc = roc_auc_score(y_test, y_proba)
print(f"Test set ROC AUC score: {auc:.4f}")

Test set ROC AUC score: 0.6503


In [64]:
class_counts = np.bincount(y_test)
print(f"Class counts in test set: {class_counts}")

Class counts in test set: [2934   13]


In [65]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [66]:
grid_search.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200, 500]},
             scoring='roc_auc')

In [67]:
best_model = grid_search.best_estimator_

In [68]:
y_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
auc = roc_auc_score(y_test, y_proba)
print(f"Test set ROC AUC score: {auc:.4f}")

Test set ROC AUC score: 0.5570
